In [4]:
## Query real time data from DingXiangYuan, and keep the latest records every day for each city

## Created on Sat Feb  8 12:41:50 2020
## Author: leebond
#### resource: https://github.com/jianxu305/nCov2019_analysis

import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import numpy as np
import math
import datetime
import warnings
warnings.filterwarnings('ignore')
from googletrans import Translator # package used to translate Chinese into English
translator = Translator()

In [6]:
## Query the latest data
! python dataset.py

2020-02-12 21:21:25数据更新成功


In [7]:
DXYArea = pd.read_csv('../data/DXY_Chinese.csv') # Read Chinese version 
# select column
DXYArea = DXYArea[['date','country','countryCode','province', 'city', 'confirmed', 'suspected', 'cured', 'dead']]

DXYArea.tail(2)  

,date,country,countryCode,province,city,confirmed,suspected,cured,dead
9456,2020-02-12,美国,US,NaN,NaN,13,0,3,0
9457,2020-02-12,越南,VN,NaN,NaN,15,0,3,0


In [9]:
DXYArea[(DXYArea['countryCode'] == 'CN') & (DXYArea['province'].isnull())].tail()

,date,country,countryCode,province,city,confirmed,suspected,cured,dead
6755,2020-02-08,中国,CN,NaN,NaN,37162,28942,2651,812
7293,2020-02-09,中国,CN,NaN,NaN,40224,23589,3283,909
7832,2020-02-10,中国,CN,NaN,NaN,42708,21675,3998,1017
8375,2020-02-11,中国,CN,NaN,NaN,44730,16067,4742,1114
8919,2020-02-12,中国,CN,NaN,NaN,44763,16067,4940,1115


In [11]:
def isNaN(num):
    return num != num

## Resoruce for Chinese - English Translation
with open('chineseProvince_to_EN.pkl','rb') as f:
    prov_dict = pkl.load(f)

    
with open('chineseCity_to_EN.pkl','rb') as f:
    city_dict = pkl.load(f)    
        
def translate_to_English(data, prov_dict, city_dict):
    """Translate Chinese in dataset to English
    """        
    data['province'] = data['province'].apply(getProvinceTranslation)
    data['city'] = data['city'].apply(getCityTranslation)
    
    for city in unable_translation: # remove these unable translated data
        data = data[data['city']!=city]
    return data
    
def getProvinceTranslation(name):
    if not isNaN(name):
        return prov_dict[name]
    else: 
        return name

unable_translation = []
def getCityTranslation(name):
    try:
        if not isNaN(name): 
            return city_dict[name]
        else:
            return name
    except:
        unable_translation.append(name)
        #print(name + ' cannot be translated\n')
        return name
    

In [12]:
daily_frm_DXYArea = translate_to_English(DXYArea, prov_dict, city_dict)

daily_frm_DXYArea.head(3)

,date,country,countryCode,province,city,confirmed,suspected,cured,dead
0,2019-12-01,中国,CN,NaN,NaN,1,0,0,0
1,2019-12-01,中国,CN,Hubei Province,NaN,1,0,0,0
2,2019-12-01,中国,CN,Hubei Province,Wuhan,1,0,0,0


In [17]:
daily_frm_DXYArea[(daily_frm_DXYArea['countryCode'] == 'CN') & (daily_frm_DXYArea['province'].isnull())].tail()

,date,country,countryCode,province,city,confirmed,suspected,cured,dead,Days,net_confirmed
6755,2020-02-08,中国,CN,NaN,NaN,37162,28942,2651,812,62,33699
7293,2020-02-09,中国,CN,NaN,NaN,40224,23589,3283,909,63,36032
7832,2020-02-10,中国,CN,NaN,NaN,42708,21675,3998,1017,64,37693
8375,2020-02-11,中国,CN,NaN,NaN,44730,16067,4742,1114,65,38874
8919,2020-02-12,中国,CN,NaN,NaN,44763,16067,4940,1115,66,38708


In [14]:
def add_days(DXYArea: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """
    Create a new column: Days, number of days after 2019-12-08 (detect the first case)
    """
    DXYArea['date'] = pd.to_datetime(DXYArea['date'])
    first_day = datetime.datetime(2019, 12, 8) # the time when detected the first case (2019-12-08)
    DXYArea['Days'] = (DXYArea['date'] - first_day).dt.days
    return DXYArea

def add_net_confirmed_case(DXYArea: pd.core.frame.DataFrame)-> pd.core.frame.DataFrame:
    """
    Add net confirmed case = confirmed - cured - dead
    """
    DXYArea['net_confirmed'] = DXYArea['confirmed'] - DXYArea['cured'] - DXYArea['dead']
    return DXYArea

In [15]:
daily_frm_DXYArea = add_days(daily_frm_DXYArea)  # add the number of days after 2019-12-08
daily_frm_DXYArea = add_net_confirmed_case(daily_frm_DXYArea) # add net confirmed case

In [16]:
daily_frm_DXYArea.tail()

,date,country,countryCode,province,city,confirmed,suspected,cured,dead,Days,net_confirmed
9453,2020-02-12,瑞典,SE,NaN,NaN,1,0,0,0,66,1
9454,2020-02-12,新加坡,SG,NaN,NaN,47,0,9,0,66,38
9455,2020-02-12,泰国,TH,NaN,NaN,33,0,10,0,66,23
9456,2020-02-12,美国,US,NaN,NaN,13,0,3,0,66,10
9457,2020-02-12,越南,VN,NaN,NaN,15,0,3,0,66,12


In [18]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)